<a href="https://colab.research.google.com/github/aaryak289/nlpJ044/blob/master/ClassAssignment/ClassworkAssignment4_J044.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd # Importing and downloading the packages and downloading 
import gzip
import nltk
from sklearn.metrics import confusion_matrix, accuracy_score
nltk.download('punkt')
from textblob import TextBlob
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
url = 'https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv' # twitter data
df = pd.read_csv(url, error_bad_lines=False)
df.head()
df = df.iloc[0:100,:]

In [0]:
df_1= pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz" , lines=True) #data for review text
df_1.head()
df_1 = df_1.iloc[0:100,:]

In [0]:
def textblobfunction(text):  #creating function for textblob
  blob = TextBlob(text)
  for sentence in blob.sentences:
     print(sentence.sentiment.polarity)
  return text

In [37]:
textblobfunction(df_1['reviewText'][0]) # running textblob for the first row

0.25
-0.5
0.0
0.0
0.41875
0.19166666666666665
0.0
0.25
0.07500000000000001
0.0


"I bought my first HP12C in about 1984 or so, and it served me faithfully until 2002 when I lost it while travelling.  I searched for another one to replace it, but found one difficult to come by in my area.  So, I decided to buy up and purchased an HP 49G.  What a mistake!  I know that many people view the HP 49G (now 49G+) as the flagship of the HP line, but for me that was a disaster.The 49G may be powerful, but using it is extremely counterintuitive...and the manual was sketchy at best.  The 12C, on the other hand, does what I need in a way that makes good sense to me.If you are looking for a solid, reliable calculator, the HP12C may be for you.  It's programmable.  It does basic statistics well, and many business applications too.  The manual makes sense; you will be up and running in short order.I'm ready to set my 49G aside and move back to my old friend.  I didn't even have to replace the batteries in well over a decade of use!HP 12C, I'm coming home!"

In [0]:
def TEXTBLOB(text):     # function to check the polarity
  TEXTBLOB = TextBlob(text).sentiment
  polarity = TEXTBLOB.polarity
  if polarity == 0:
    return "neutral"
  elif polarity > 0:
     return "positive"
  else:
    return "negative"

In [39]:
df_1['reviewText_textblob'] = df_1['reviewText'].apply(TEXTBLOB) #applying textblob on review text
df_1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewText_textblob
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004",positive
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007",positive
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011",positive
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006",positive
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013",positive


In [40]:
df['TweetText_textblob'] = df['TweetText'].apply(TEXTBLOB) #applying textblob on twitter text
df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,TweetText_textblob
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,positive


In [0]:
def sent_int_analyzer(text):       #using sentiment ananlyzer to predict the sentiments
  sia = SentimentIntensityAnalyzer()
  sentences = text
  for sentence in sentences:
    score = sia.polarity_scores(sentences)
    if score['compound'] <= -0.05:
      return "negative"
    elif score['compound'] >= 0.05:
      return "positive"
    else:
      return "neutral"

In [42]:
df_1['reviewText_intensity_analyzer'] = df_1['reviewText'].apply(sent_int_analyzer) #applying sentiment analyzer on review text
df_1.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewText_textblob,reviewText_intensity_analyzer
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004",positive,positive
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007",positive,positive
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011",positive,negative
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006",positive,positive
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013",positive,positive


In [43]:
df['TweetText_intensity_analyzer'] = df['TweetText'].apply(sent_int_analyzer) # applying sentiment analyzer on twitter
df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,TweetText_textblob,TweetText_intensity_analyzer
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,neutral,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,positive,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,positive,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,positive,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,positive,positive


In [44]:
df = df[df['Sentiment'] != "irrelevant"]
df['Sentiment'].value_counts()

positive    100
Name: Sentiment, dtype: int64

In [45]:
print(confusion_matrix(df['Sentiment'],df['TweetText_textblob']))  #creating confusion matrix and obtaining accuracy score
print(accuracy_score(df['Sentiment'],df['TweetText_textblob']))
print(confusion_matrix(df['Sentiment'],df['TweetText_intensity_analyzer']))
print(accuracy_score(df['Sentiment'],df['TweetText_intensity_analyzer']))

[[ 0  0  0]
 [ 0  0  0]
 [ 5 23 72]]
0.72
[[ 0  0  0]
 [ 0  0  0]
 [ 9 15 76]]
0.76
